In [15]:
import pygame
import random

def Background_color(surface):
    for y in range(screen_height):
        for x in range(screen_width):
            ratio = (x - y + screen_width) / (2 * screen_width)
            r = int(WHITE[0] + (ADDFFF[0] - WHITE[0]) * ratio)
            g = int(WHITE[1] + (ADDFFF[1] - WHITE[1]) * ratio)
            b = int(WHITE[2] + (ADDFFF[2] - WHITE[2]) * ratio)
            surface.set_at((x,y), (r,g,b))

class Menu():
    def __init__(self, x, y, width, height, menu_state):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.options = []
        self.selected_option = 0

        self.menu_state = menu_state
        self.current_menu = "mode"
        self.current_option = 0

    def add_option(self, option):
        self.options.append(option)

    def draw(self, window):
        title_font = pygame.font.Font(None, 60)
        font = pygame.font.Font(None, 36)

        title_text = title_font.render("PongGame", True, CADETBLUE)
        title_rect = title_text.get_rect(center = (screen_width//2, screen_height//3))
        window.blit(title_text, title_rect)
        for i, option in enumerate(self.options):
            if self.current_option == i:
                pygame.draw.rect(window, CADETBLUE, 
                                 (self.x, 
                                  self.y + (i + 1) * (self.height // (len(self.options) + 1)) - 15,
                                  self.width, 30), 
                                  3)
            text = font.render(option, True, CADETBLUE)
            text_rect = text.get_rect(center=(self.x + self.width // 2, self.y + (i + 1) * (self.height // (len(self.options) + 1)) - 15 + 15))
            window.blit(text, text_rect)


    def handle_mouse_input(self, x, y):
        for i, option in enumerate(self.options):
            if self.x <= x <= self.x + self.width and self.y + (i + 1) * (self.height // (len(self.options) + 1)) <= y <= self.y + (i + 2) * (self.height // (len(self.options) + 1)):
                return i
        return None

    def handle_keyboard_input(self, keys):
        option_change = 0
        if keys[pygame.K_UP]:
            option_change = -1
        elif keys[pygame.K_DOWN]:
            option_change = 1

        self.current_option += option_change
        if self.current_option < 0:
            self.current_option = len(self.options) - 1
        elif self.current_option >= len(self.options):
            self.current_option = 0

        return option_change
    
    def handle_events(self, keys):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return False, True
            
            # Get mouse event
            elif event.type == pygame.MOUSEBUTTONDOWN:
                x, y = pygame.mouse.get_pos()
                if self.current_menu == self.menu:
                    selected_option = self.handle_mouse_input(x, y)
                    if selected_option is not None:
                        if selected_option == 0:
                            self.current_menu = "difficulty"
                        elif selected_option == 1:
                            self.start_game()
                elif self.current_menu == "difficulty":
                    selected_option = self.handle_mouse_input(x, y)
                    if selected_option is not None:
                        if selected_option == 0:
                            self.start_game(difficulty="easy")
                        elif selected_option == 1:
                            self.start_game(difficulty="medium")
                        elif selected_option == 2:
                            self.start_game(difficulty="hard")

            # Get keyboard event
            elif event.type == pygame.KEYDOWN:
                if self.current_menu == self.menu:
                    option_change = self.handle_keyboard_input(keys)
                    self.current_option += option_change
                    if self.current_option < 0:
                        self.current_option = len(self.options) - 1
                    elif self.current_option >= len(self.options):
                        self.current_option = 0
                    if keys[pygame.K_RETURN]:
                        if self.current_option == 0:
                            self.current_menu = "difficulty"
                        elif self.current_option == 1:
                            self.start_game()
                elif self.current_menu == "difficulty":
                    option_change = self.handle_keyboard_input(keys)
                    if option_change != 0:
                        self.current_option += option_change
                        if self.current_option < 0:
                            self.current_option = len(self.options) - 1
                        elif self.current_option >= len(self.options):
                            self.current_option = 0
                    if keys[pygame.K_RETURN]:
                        if self.current_option == 0:
                            self.start_game(difficulty="easy")
                        elif self.current_option == 1:
                            self.start_game(difficulty="medium")
                        elif self.current_option == 2:
                            self.start_game(difficulty="hard")
        return False, True
    
    def start_game(self, difficulty):
        return

class Ball():
    def __init__(self, x, y, radius, score):
        self.x = x
        self.y = y
        self.x_direction = random.choice((-3, 3))
        self.y_direction = random.choice([-3, 3])
        self.radius = radius
        self.score = score
    
    def move(self):
        self.x += self.x_direction
        self.y += self.y_direction
        self.out_of_bound()
    
    def bounce(self, brick):
        if self.x < brick.x - (brick.radius / 1.414):
            self.x_direction = -2
        elif self.x > brick.x + (brick.radius / 1.414):
            self.x_direction = 2
        elif self.y < brick.y - (brick.radius / 1.414):
            self.y_direction = -2
        elif self.y > brick.y + (brick.radius / 1.414):
            self.y_direction = 2
    
    def contact_detect_wall(self, wall):
        if self.y - wall.height - self.radius  <= 0 or self.y >= screen_height - wall.height - self.radius:
            self.y_direction *= -1
    
    def contact_detect_pad1(self, pad):
        if self.x == pad.x + pad.width + self.radius and pad.y - self.radius <= self.y <= pad.y + pad.height + self.radius:
            self.x_direction = BALL_SPEED_X

    def contact_detect_pad2(self, pad):        
        if self.x == pad.x - self.radius and pad.y <= self.y <= pad.y + pad.height:
            self.x_direction = -BALL_SPEED_X

    def out_of_bound(self):
        if self.x < 0:
            self.x = screen_width // 2
            self.y = screen_height * (random.choice([0.25, 0.375, 0.5, 0.625, 0.75]))
            self.x_direction *= random.choice([-1, 1])
            self.y_direction *= random.choice([-1, 1])
            self.score.player2_score += 1
        elif self.x + self.radius > screen_width:
            self.x = screen_width // 2
            self.y = screen_height * (random.choice([0.25, 0.375, 0.5, 0.625, 0.75]))
            self.x_direction *= random.choice([-1, 1])
            self.y_direction *= random.choice([-1, 1])
            self.score.player1_score += 1

    def contact_detect_brick(self, brick):
        if (brick.dis_to_brick(self.x, self.y) - self.radius <= 0) and brick.valid:
            self.bounce(brick)
            brick.valid = False
    
    def draw(self, window):
        pygame.draw.circle(window, DEEPSKYBLUE, (self.x, self.y), self.radius)

    def reset(self, x, y, x_direction, y_direction):
        self.x = x
        self.y = y
        self.x_direction = random.choice([-3, 3])
        self.y_direction = random.choice([-3, 3])

class Paddle1():
    def __init__(self, x, y, width, height):
        self.x = x
        self.y = y
        self.width = width
        self.height = height

    def draw(self, window):
        pygame.draw.rect(window, DEEPSKYBLUE, (self.x, self.y, self.width, self.height))

    def move(self, keys):
        if keys[pygame.K_w] and self.y - WALL_HEIGHT > 0:
            self.y -= PADDLE_SPEED
        if keys[pygame.K_s] and self.y < screen_height - self.height - WALL_HEIGHT:
            self.y += PADDLE_SPEED

class Paddle2(Paddle1):
    def move(self, keys):
        if keys[pygame.K_UP] and self.y - WALL_HEIGHT > 0:
            self.y -= PADDLE_SPEED
        if keys[pygame.K_DOWN] and self.y < screen_height - self.height - WALL_HEIGHT:
            self.y += PADDLE_SPEED

class Wall1():
    def __init__(self, x, y, width, height):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
    def draw(self, window):
        pygame.draw.rect(window, CADETBLUE, (0, 0, WALL_WIDTH, WALL_HEIGHT))

class Wall2(Wall1):
    def draw(self, window):
        pygame.draw.rect(window, CADETBLUE, (0, screen_height - WALL_HEIGHT, WALL_WIDTH, WALL_HEIGHT))

class Score():
    def __init__(self, player1_score, player2_score):
        self.player1_score = player1_score
        self.player2_score = player2_score

    def draw(self, window):
        font = pygame.font.Font(None, 45)
        player1_score_text = font.render(str(self.player1_score), True, CADETBLUE)
        player1_score_rect = player1_score_text.get_rect(center=(screen_width // 4, 50))
        window.blit(player1_score_text, player1_score_rect)

        player2_score_text = font.render(str(self.player2_score), True, CADETBLUE)
        player2_score_rect = player2_score_text.get_rect(center=(3 * screen_width // 4, 50))
        window.blit(player2_score_text, player2_score_rect)

# Initialize Pygame
pygame.init()

# Define colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
CYAN =  (0, 255, 255)
ADDFFF = (173, 233, 255)
DEEPSKYBLUE = (0,191,255)
CADETBLUE = (95, 158, 160)

# Define game constants
BALL_RADIUS = 10
PADDLE_WIDTH = 10
PADDLE_HEIGHT = 60
BALL_SPEED_X = 3
BALL_SPEED_Y = 3
PADDLE_SPEED = 5
PLAYER1_SCORE = 0
PLAYER2_SCORE = 0

# Define game strings
Player1_name = "Player1"
Player2_name = "Player2"

# Set up the game window
screen_width = 800
screen_height = 400
window = pygame.display.set_mode((screen_width, screen_height))

# Set title
pygame.display.set_caption("Pong")

# Set player mode menu
Menu_state = "mode"
menu = Menu(300, 200, 200, 100, Menu_state)
menu.add_option("Single Player")
menu.add_option("Multiplayer")

# Set difficulty menu
difficulty_menu = Menu(300, 200, 200, 100, "difficulty")
difficulty_menu.add_option("Easy")
difficulty_menu.add_option("Medium")
difficulty_menu.add_option("Hard")

# Set up score
score = Score(PLAYER1_SCORE, PLAYER2_SCORE)

# Set up wall
WALL_WIDTH = screen_width
WALL_HEIGHT = 10
wall1 = Wall1(0,0, WALL_WIDTH, WALL_HEIGHT)
wall2 = Wall2(0, screen_height - WALL_HEIGHT, WALL_WIDTH, WALL_HEIGHT)

# Set up ball
ball_x = screen_width // 2
ball_y = screen_height // 2

ball = Ball(ball_x, ball_y, BALL_RADIUS, score)

# Set up paddles
paddle1_x = 20
paddle1_y = screen_height // 2 - PADDLE_HEIGHT // 2
paddle2_x = screen_width - 20 - PADDLE_WIDTH
paddle2_y = screen_height // 2 - PADDLE_HEIGHT // 2

pad1 = Paddle1(paddle1_x, paddle1_y, PADDLE_WIDTH, PADDLE_HEIGHT)
pad2 = Paddle2(paddle2_x, paddle2_y, PADDLE_WIDTH, PADDLE_HEIGHT)

# Create the background surface
background_surface = pygame.Surface((screen_width, screen_height))
Background_color(background_surface)

# Create the clock object to control the frame rate
clock = pygame.time.Clock()

# Menu loop
while Menu_running:
    window.blit(background_surface, (0, 0))

    if menu.current_menu == "mode":
        menu.draw(window)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                Menu_running = False
            else:
                selected_option = menu.handle_events(event)
                if selected_option is not None:
                    if selected_option == 0:
                        menu.current_menu = "difficulty"
                    elif selected_option == 1:
                        print("Start multiplayer game")
                        Menu_running = False
    elif menu.current_menu == "difficulty":
        difficulty_menu.draw(window)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                Menu_running = False
            else:
                selected_option = difficulty_menu.handle_events(event)
                if selected_option is not None:
                    if selected_option == 0:
                        print("Start single player game with easy difficulty")
                        Menu_running = False
                    elif selected_option == 1:
                        print("Start single player game with medium difficulty")
                        Menu_running = False
                    elif selected_option == 2:
                        print("Start single player game with hard difficulty")
                        Menu_running = False

    pygame.display.update()
    clock.tick(60)


while running:
    # Clear the screen
    window.blit(background_surface, (0, 0))

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Draw walls
    wall1.draw(window)
    wall2.draw(window)

    # Draw the ball
    ball.move()
    ball.draw(window)

    # Draw the paddle
    keys = pygame.key.get_pressed()
    pad1.move(keys)
    pad1.draw(window)
    pad2.move(keys)
    pad2.draw(window)

    # Detect contact walls & paddles
    ball.contact_detect_wall(wall1)
    ball.contact_detect_pad1(pad1)
    ball.contact_detect_pad2(pad2)

    # Record score
    score.draw(window)

    # Update the window
    pygame.display.flip()
    
    # Limit the frame rate
    clock.tick(60)

pygame.quit()

KeyboardInterrupt: 

In [13]:
import pygame
import random

def Background_color(surface):
    for y in range(screen_height):
        for x in range(screen_width):
            ratio = (x - y + screen_width) / (2 * screen_width)
            r = int(WHITE[0] + (ADDFFF[0] - WHITE[0]) * ratio)
            g = int(WHITE[1] + (ADDFFF[1] - WHITE[1]) * ratio)
            b = int(WHITE[2] + (ADDFFF[2] - WHITE[2]) * ratio)
            surface.set_at((x,y), (r,g,b))

class Menu():
    def __init__(self, x, y, width, height, menu_state):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.options = []
        self.selected_option = 0

        self.menu = menu_state
        self.current_menu = "mode"
        self.current_option = 0

    def add_option(self, option):
        self.options.append(option)

    def draw(self, window):
        title_font = pygame.font.Font(None, 60)
        font = pygame.font.Font(None, 36)

        title_text = title_font.render("PongGame", True, CADETBLUE)
        title_rect = title_text.get_rect(center = (screen_width//2, screen_height//3))
        window.blit(title_text, title_rect)
        for i, option in enumerate(self.options):
            if self.current_option == i:
                pygame.draw.rect(window, CADETBLUE, 
                                 (self.x, 
                                  self.y + (i + 1) * (self.height // (len(self.options) + 1)) - 15,
                                  self.width, 30), 
                                  3)
            text = font.render(option, True, CADETBLUE)
            text_rect = text.get_rect(center=(self.x + self.width // 2, self.y + (i + 1) * (self.height // (len(self.options) + 1)) - 15 + 15))
            window.blit(text, text_rect)

    def handle_mouse_input(self, x, y):
        for i, option in enumerate(self.options):
            if self.x <= x <= self.x + self.width and self.y + (i + 1) * (self.height // (len(self.options) + 1)) <= y <= self.y + (i + 2) * (self.height // (len(self.options) + 1)):
                return i
        return None

    def handle_keyboard_input(self, keys):
        option_change = 0
        if keys[pygame.K_UP]:
            option_change = -1
        elif keys[pygame.K_DOWN]:
            option_change = 1

        self.current_option += option_change
        if self.current_option < 0:
            self.current_option = len(self.options) - 1
        elif self.current_option >= len(self.options):
            self.current_option = 0

        return option_change
    
    def handle_events(self, keys):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return False, True
            
            # Get mouse event
            elif event.type == pygame.MOUSEBUTTONDOWN:
                x, y = pygame.mouse.get_pos()
                if self.current_menu == self.menu:
                    selected_option = self.handle_mouse_input(x, y)
                    if selected_option is not None:
                        if selected_option == 0:
                            self.current_menu = "difficulty"
                        elif selected_option == 1:
                            self.start_game()
                elif self.current_menu == "difficulty":
                    selected_option = self.handle_mouse_input(x, y)
                    if selected_option is not None:
                        if selected_option == 0:
                            self.start_game(difficulty="easy")
                        elif selected_option == 1:
                            self.start_game(difficulty="medium")
                        elif selected_option == 2:
                            self.start_game(difficulty="hard")

            # Get keyboard event
            elif event.type == pygame.KEYDOWN:
                if self.current_menu == self.menu:
                    option_change = self.handle_keyboard_input(keys)
                    self.current_option += option_change
                    if self.current_option < 0:
                        self.current_option = len(self.options) - 1
                    elif self.current_option >= len(self.options):
                        self.current_option = 0
                    if keys[pygame.K_RETURN]:
                        if self.current_option == 0:
                            self.current_menu = "difficulty"
                        elif self.current_option == 1:
                            self.start_game()
                elif self.current_menu == "difficulty":
                    option_change = self.handle_keyboard_input(keys)
                    if option_change != 0:
                        self.current_option += option_change
                        if self.current_option < 0:
                            self.current_option = len(self.options) - 1
                        elif self.current_option >= len(self.options):
                            self.current_option = 0
                    if keys[pygame.K_RETURN]:
                        if self.current_option == 0:
                            self.start_game(difficulty="easy")
                        elif self.current_option == 1:
                            self.start_game(difficulty="medium")
                        elif self.current_option == 2:
                            self.start_game(difficulty="hard")
            return False, True
    
    def start_game(self, difficulty):
        return

class Ball():
    def __init__(self, x, y, radius, score):
        self.x = x
        self.y = y
        self.x_direction = random.choice((-3, 3))
        self.y_direction = random.choice([-3, 3])
        self.radius = radius
        self.score = score
    
    def move(self):
        self.x += self.x_direction
        self.y += self.y_direction
        self.out_of_bound()
    
    def bounce(self, brick):
        if self.x < brick.x - (brick.radius / 1.414):
            self.x_direction = -2
        elif self.x > brick.x + (brick.radius / 1.414):
            self.x_direction = 2
        elif self.y < brick.y - (brick.radius / 1.414):
            self.y_direction = -2
        elif self.y > brick.y + (brick.radius / 1.414):
            self.y_direction = 2
    
    def contact_detect_wall(self, wall):
        if self.y - wall.height - self.radius  <= 0 or self.y >= screen_height - wall.height - self.radius:
            self.y_direction *= -1
    
    def contact_detect_pad1(self, pad):
        if self.x == pad.x + pad.width + self.radius and pad.y - self.radius <= self.y <= pad.y + pad.height + self.radius:
            self.x_direction = BALL_SPEED_X

    def contact_detect_pad2(self, pad):        
        if self.x == pad.x - self.radius and pad.y <= self.y <= pad.y + pad.height:
            self.x_direction = -BALL_SPEED_X

    def out_of_bound(self):
        if self.x < 0:
            self.x = screen_width // 2
            self.y = screen_height * (random.choice([0.25, 0.375, 0.5, 0.625, 0.75]))
            self.x_direction *= random.choice([-1, 1])
            self.y_direction *= random.choice([-1, 1])
            self.score.player2_score += 1
        elif self.x + self.radius > screen_width:
            self.x = screen_width // 2
            self.y = screen_height * (random.choice([0.25, 0.375, 0.5, 0.625, 0.75]))
            self.x_direction *= random.choice([-1, 1])
            self.y_direction *= random.choice([-1, 1])
            self.score.player1_score += 1

    def contact_detect_brick(self, brick):
        if (brick.dis_to_brick(self.x, self.y) - self.radius <= 0) and brick.valid:
            self.bounce(brick)
            brick.valid = False
    
    def draw(self, window):
        pygame.draw.circle(window, DEEPSKYBLUE, (self.x, self.y), self.radius)

    def reset(self, x, y, x_direction, y_direction):
        self.x = x
        self.y = y
        self.x_direction = random.choice([-3, 3])
        self.y_direction = random.choice([-3, 3])

class Paddle1():
    def __init__(self, x, y, width, height):
        self.x = x
        self.y = y
        self.width = width
        self.height = height

    def draw(self, window):
        pygame.draw.rect(window, DEEPSKYBLUE, (self.x, self.y, self.width, self.height))

    def move(self, keys):
        if keys[pygame.K_w] and self.y - WALL_HEIGHT > 0:
            self.y -= PADDLE_SPEED
        if keys[pygame.K_s] and self.y < screen_height - self.height - WALL_HEIGHT:
            self.y += PADDLE_SPEED

class Paddle2(Paddle1):
    def move(self, keys):
        if keys[pygame.K_UP] and self.y - WALL_HEIGHT > 0:
            self.y -= PADDLE_SPEED
        if keys[pygame.K_DOWN] and self.y < screen_height - self.height - WALL_HEIGHT:
            self.y += PADDLE_SPEED

class Wall1():
    def __init__(self, x, y, width, height):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
    def draw(self, window):
        pygame.draw.rect(window, CADETBLUE, (0, 0, WALL_WIDTH, WALL_HEIGHT))

class Wall2(Wall1):
    def draw(self, window):
        pygame.draw.rect(window, CADETBLUE, (0, screen_height - WALL_HEIGHT, WALL_WIDTH, WALL_HEIGHT))

class Score():
    def __init__(self, player1_score, player2_score):
        self.player1_score = player1_score
        self.player2_score = player2_score

    def draw(self, window):
        font = pygame.font.Font(None, 45)
        player1_score_text = font.render(str(self.player1_score), True, CADETBLUE)
        player1_score_rect = player1_score_text.get_rect(center=(screen_width // 4, 50))
        window.blit(player1_score_text, player1_score_rect)

        player2_score_text = font.render(str(self.player2_score), True, CADETBLUE)
        player2_score_rect = player2_score_text.get_rect(center=(3 * screen_width // 4, 50))
        window.blit(player2_score_text, player2_score_rect)

# Initialize Pygame
pygame.init()

# Define colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
CYAN =  (0, 255, 255)
ADDFFF = (173, 233, 255)
DEEPSKYBLUE = (0,191,255)
CADETBLUE = (95, 158, 160)

# Define game constants
BALL_RADIUS = 10
PADDLE_WIDTH = 10
PADDLE_HEIGHT = 60
BALL_SPEED_X = 3
BALL_SPEED_Y = 3
PADDLE_SPEED = 5
PLAYER1_SCORE = 0
PLAYER2_SCORE = 0

# Define game strings
Player1_name = "Player1"
Player2_name = "Player2"

# Set up the game window
screen_width = 800
screen_height = 400
window = pygame.display.set_mode((screen_width, screen_height))

# Set title
pygame.display.set_caption("Pong")

# Set player mode menu
Menu_state = "mode"
menu = Menu(300, 200, 200, 100, Menu_state)
menu.add_option("Single Player")
menu.add_option("Multiplayer")

# Set difficulty menu
difficulty_menu = Menu(300, 200, 200, 100, "difficulty")
difficulty_menu.add_option("Easy")
difficulty_menu.add_option("Medium")
difficulty_menu.add_option("Hard")

# Set up score
score = Score(PLAYER1_SCORE, PLAYER2_SCORE)

# Set up wall
WALL_WIDTH = screen_width
WALL_HEIGHT = 10
wall1 = Wall1(0,0, WALL_WIDTH, WALL_HEIGHT)
wall2 = Wall2(0, screen_height - WALL_HEIGHT, WALL_WIDTH, WALL_HEIGHT)

# Set up ball
ball_x = screen_width // 2
ball_y = screen_height // 2

ball = Ball(ball_x, ball_y, BALL_RADIUS, score)

# Set up paddles
paddle1_x = 20
paddle1_y = screen_height // 2 - PADDLE_HEIGHT // 2
paddle2_x = screen_width - 20 - PADDLE_WIDTH
paddle2_y = screen_height // 2 - PADDLE_HEIGHT // 2

pad1 = Paddle1(paddle1_x, paddle1_y, PADDLE_WIDTH, PADDLE_HEIGHT)
pad2 = Paddle2(paddle2_x, paddle2_y, PADDLE_WIDTH, PADDLE_HEIGHT)

# Create the background surface
background_surface = pygame.Surface((screen_width, screen_height))
Background_color(background_surface)

# Create the clock object to control the frame rate
clock = pygame.time.Clock()

# Menu loop
Menu_running = True
while Menu_running:
    window.blit(background_surface, (0, 0))

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            Menu_running = False

    if menu.current_menu == "mode":
        # 繪製菜單和處理事件
        menu.draw(window)
        menu_keys = pygame.key.get_pressed()
        Menu_running, running = menu.handle_events(menu_keys)

    elif menu.current_menu == "difficulty":
        # 繪製難度選單和處理事件
        difficulty_menu.draw(window)
        difficulty_keys = pygame.key.get_pressed()
        Menu_running, running = difficulty_menu.handle_events(difficulty_keys)

    pygame.display.flip()
    clock.tick(60)

while running:
    # Clear the screen
    window.blit(background_surface, (0, 0))

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Draw walls
    wall1.draw(window)
    wall2.draw(window)

    # Draw the ball
    ball.move()
    ball.draw(window)

    # Draw the paddle
    keys = pygame.key.get_pressed()
    pad1.move(keys)
    pad1.draw(window)
    pad2.move(keys)
    pad2.draw(window)

    # Detect contact walls & paddles
    ball.contact_detect_wall(wall1)
    ball.contact_detect_pad1(pad1)
    ball.contact_detect_pad2(pad2)

    # Record score
    score.draw(window)

    # Update the window
    pygame.display.flip()
    
    # Limit the frame rate
    clock.tick(60)

pygame.quit()

TypeError: cannot unpack non-iterable NoneType object

In [5]:
import pygame
import random

def Background_color(surface):
    for y in range(screen_height):
        for x in range(screen_width):
            ratio = (x - y + screen_width) / (2 * screen_width)
            r = int(WHITE[0] + (ADDFFF[0] - WHITE[0]) * ratio)
            g = int(WHITE[1] + (ADDFFF[1] - WHITE[1]) * ratio)
            b = int(WHITE[2] + (ADDFFF[2] - WHITE[2]) * ratio)
            surface.set_at((x,y), (r,g,b))

class Menu():
    # def __init__(self, x, y, width, height, menu_state):
    def __init__(self, x, y, width, height):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.options = []
        self.selected_option = 0

        # self.menu = menu_state
        # self.current_menu = self.menu
        # self.current_option = 0
        self.menu = Menu(300, 200, 200, 100)
        self.menu.add_option("Single Player")
        self.menu.add_option("Multiplayer")

        self.difficulty_menu = Menu(300, 200, 200, 100)
        self.difficulty_menu.add_option("Easy")
        self.difficulty_menu.add_option("Medium")
        self.difficulty_menu.add_option("Hard")

        self.current_menu = self.menu
        self.current_option = 0

    def add_option(self, option):
        self.options.append(option)

    def draw(self, window):
        title_font = pygame.font.Font(None, 60)
        font = pygame.font.Font(None, 36)

        title_text = title_font.render("PongGame", True, CADETBLUE)
        title_rect = title_text.get_rect(center = (screen_width//2, screen_height//3))
        window.blit(title_text, title_rect)
        for i, option in enumerate(self.options):
            if self.current_option == i:
                pygame.draw.rect(window, CADETBLUE, 
                                 (self.x, 
                                  self.y + (i + 1) * (self.height // (len(self.options) + 1)) - 15,
                                  self.width, 30), 
                                  3)
            text = font.render(option, True, CADETBLUE)
            text_rect = text.get_rect(center=(self.x + self.width // 2, self.y + (i + 1) * (self.height // (len(self.options) + 1))))
            window.blit(text, text_rect)

    def handle_mouse_input(self, x, y):
        for i, option in enumerate(self.options):
            if self.x <= x <= self.x + self.width and self.y + (i + 1) * (self.height // (len(self.options) + 1)) <= y <= self.y + (i + 2) * (self.height // (len(self.options) + 1)):
                return i
        return None

    def handle_keyboard_input(self, keys):
        if keys[pygame.K_UP]:
            return -1
        elif keys[pygame.K_DOWN]:
            return 1
        return 0
    
    # def handle_events(self, keys):
    #     for event in pygame.event.get():
    #         if event.type == pygame.QUIT:
    #             return False, False
            
    #         # Get mouse event
    #         elif event.type == pygame.MOUSEBUTTONDOWN:
    #             x, y = pygame.mouse.get_pos()
    #             if self.current_menu == self.menu:
    #                 selected_option = self.handle_mouse_input(x, y)
    #                 if selected_option is not None:
    #                     if selected_option == 0:
    #                         self.current_menu = "difficulty"
    #                     elif selected_option == 1:
    #                         self.start_game()
    #             elif self.current_menu == "difficulty":
    #                 selected_option = self.handle_mouse_input(x, y)
    #                 if selected_option is not None:
    #                     if selected_option == 0:
    #                         self.start_game(difficulty="easy")
    #                     elif selected_option == 1:
    #                         self.start_game(difficulty="medium")
    #                     elif selected_option == 2:
    #                         self.start_game(difficulty="hard")

    #         # Get keyboard event
    #         elif event.type == pygame.KEYDOWN:
    #             if self.current_menu == self.menu:
    #                 option_change = self.handle_keyboard_input(keys)
    #                 self.current_option += option_change
    #                 if self.current_option < 0:
    #                     self.current_option = len(self.options) - 1
    #                     # print("1")
    #                 elif self.current_option >= len(self.options):
    #                     self.current_option = 0
    #                 if keys[pygame.K_RETURN]:
    #                     if self.current_option == 0:
    #                         self.current_menu = "difficulty"
    #                     elif self.current_option == 1:
    #                         self.start_game()
    #             elif self.current_menu == "difficulty":
    #                 option_change = self.handle_keyboard_input(keys)
    #                 if option_change != 0:
    #                     self.current_option += option_change
    #                     if self.current_option < 0:
    #                         self.current_option = len(self.options) - 1
    #                     elif self.current_option >= len(self.options):
    #                         self.current_option = 0
    #                 if keys[pygame.K_RETURN]:
    #                     if self.current_option == 0:
    #                         self.start_game(difficulty="easy")
    #                     elif self.current_option == 1:
    #                         self.start_game(difficulty="medium")
    #                     elif self.current_option == 2:
    #                         self.start_game(difficulty="hard")
    #     return False, True
    def handle_events(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                x, y = pygame.mouse.get_pos()
                if self.current_menu == self.menu:
                    selected_option = self.current_menu.handle_mouse_input(x, y)
                    if selected_option == 0:
                        self.current_menu = self.difficulty_menu
                    elif selected_option == 1:
                        self.start_game()
                elif self.current_menu == self.difficulty_menu:
                    selected_option = self.current_menu.handle_mouse_input(x, y)
                    if selected_option == 0:
                        self.start_game(difficulty="easy")
                    elif selected_option == 1:
                        self.start_game(difficulty="medium")
                    elif selected_option == 2:
                        self.start_game(difficulty="hard")
            elif event.type == pygame.KEYDOWN:
                if self.current_menu == self.menu:
                    option_change = self.current_menu.handle_keyboard_input(pygame.key.get_pressed())
                    self.current_option += option_change
                    if self.current_option < 0:
                        self.current_option = len(self.current_menu.options) - 1
                    elif self.current_option >= len(self.current_menu.options):
                        self.current_option = 0
                    if pygame.key.get_pressed()[pygame.K_RETURN]:
                        if self.current_option == 0:
                            self.current_menu = self.difficulty_menu
                        elif self.current_option == 1:
                            self.start_game()
                elif self.current_menu == self.difficulty_menu:
                    option_change = self.current_menu.handle_keyboard_input(pygame.key.get_pressed())
                    self.current_option += option_change
                    if self.current_option < 0:
                        self.current_option = len(self.current_menu.options) - 1
                    elif self.current_option >= len(self.current_menu.options):
                        self.current_option = 0
                    if pygame.key.get_pressed()[pygame.K_RETURN]:
                        if self.current_option == 0:
                            self.start_game(difficulty="easy")
                        elif self.current_option == 1:
                            self.start_game(difficulty="medium")
                        elif self.current_option == 2:
                            self.start_game(difficulty="hard")
        return False, True
    
    def start_game(self, difficulty):
        return

class Ball():
    def __init__(self, x, y, radius, score):
        self.x = x
        self.y = y
        self.x_direction = random.choice((-3, 3))
        self.y_direction = random.choice([-3, 3])
        self.radius = radius
        self.score = score
    
    def move(self):
        self.x += self.x_direction
        self.y += self.y_direction
        self.out_of_bound()
    
    def bounce(self, brick):
        if self.x < brick.x - (brick.radius / 1.414):
            self.x_direction = -2
        elif self.x > brick.x + (brick.radius / 1.414):
            self.x_direction = 2
        elif self.y < brick.y - (brick.radius / 1.414):
            self.y_direction = -2
        elif self.y > brick.y + (brick.radius / 1.414):
            self.y_direction = 2
    
    def contact_detect_wall(self, wall):
        if self.y - wall.height - self.radius  <= 0 or self.y >= screen_height - wall.height - self.radius:
            self.y_direction *= -1
    
    def contact_detect_pad1(self, pad):
        if self.x == pad.x + pad.width + self.radius and pad.y - self.radius <= self.y <= pad.y + pad.height + self.radius:
            self.x_direction = BALL_SPEED_X

    def contact_detect_pad2(self, pad):        
        if self.x == pad.x - self.radius and pad.y <= self.y <= pad.y + pad.height:
            self.x_direction = -BALL_SPEED_X

    def out_of_bound(self):
        if self.x < 0:
            self.x = screen_width // 2
            self.y = screen_height * (random.choice([0.25, 0.375, 0.5, 0.625, 0.75]))
            self.x_direction *= random.choice([-1, 1])
            self.y_direction *= random.choice([-1, 1])
            self.score.player2_score += 1
        elif self.x + self.radius > screen_width:
            self.x = screen_width // 2
            self.y = screen_height * (random.choice([0.25, 0.375, 0.5, 0.625, 0.75]))
            self.x_direction *= random.choice([-1, 1])
            self.y_direction *= random.choice([-1, 1])
            self.score.player1_score += 1

    def contact_detect_brick(self, brick):
        if (brick.dis_to_brick(self.x, self.y) - self.radius <= 0) and brick.valid:
            self.bounce(brick)
            brick.valid = False
    
    def draw(self, window):
        pygame.draw.circle(window, DEEPSKYBLUE, (self.x, self.y), self.radius)

    def reset(self, x, y, x_direction, y_direction):
        self.x = x
        self.y = y
        self.x_direction = random.choice([-3, 3])
        self.y_direction = random.choice([-3, 3])

class Paddle1():
    def __init__(self, x, y, width, height):
        self.x = x
        self.y = y
        self.width = width
        self.height = height

    def draw(self, window):
        pygame.draw.rect(window, DEEPSKYBLUE, (self.x, self.y, self.width, self.height))

    def move(self, keys):
        if keys[pygame.K_w] and self.y - WALL_HEIGHT > 0:
            self.y -= PADDLE_SPEED
        if keys[pygame.K_s] and self.y < screen_height - self.height - WALL_HEIGHT:
            self.y += PADDLE_SPEED

class Paddle2(Paddle1):
    def move(self, keys):
        if keys[pygame.K_UP] and self.y - WALL_HEIGHT > 0:
            self.y -= PADDLE_SPEED
        if keys[pygame.K_DOWN] and self.y < screen_height - self.height - WALL_HEIGHT:
            self.y += PADDLE_SPEED

class Wall1():
    def __init__(self, x, y, width, height):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
    def draw(self, window):
        pygame.draw.rect(window, CADETBLUE, (0, 0, WALL_WIDTH, WALL_HEIGHT))

class Wall2(Wall1):
    def draw(self, window):
        pygame.draw.rect(window, CADETBLUE, (0, screen_height - WALL_HEIGHT, WALL_WIDTH, WALL_HEIGHT))

class Score():
    def __init__(self, player1_score, player2_score):
        self.player1_score = player1_score
        self.player2_score = player2_score

    def draw(self, window):
        font = pygame.font.Font(None, 45)
        player1_score_text = font.render(str(self.player1_score), True, CADETBLUE)
        player1_score_rect = player1_score_text.get_rect(center=(screen_width // 4, 50))
        window.blit(player1_score_text, player1_score_rect)

        player2_score_text = font.render(str(self.player2_score), True, CADETBLUE)
        player2_score_rect = player2_score_text.get_rect(center=(3 * screen_width // 4, 50))
        window.blit(player2_score_text, player2_score_rect)

# Initialize Pygame
pygame.init()

# Define colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
CYAN =  (0, 255, 255)
ADDFFF = (173, 233, 255)
DEEPSKYBLUE = (0,191,255)
CADETBLUE = (95, 158, 160)

# Define game constants
BALL_RADIUS = 10
PADDLE_WIDTH = 10
PADDLE_HEIGHT = 60
BALL_SPEED_X = 3
BALL_SPEED_Y = 3
PADDLE_SPEED = 5
PLAYER1_SCORE = 0
PLAYER2_SCORE = 0

# Define game strings
Player1_name = "Player1"
Player2_name = "Player2"

# Set up the game window
screen_width = 800
screen_height = 400
window = pygame.display.set_mode((screen_width, screen_height))

# Set title
pygame.display.set_caption("Pong")

# Set player mode menu
# Menu_state = "mode"
# menu = Menu(300, 200, 200, 100, Menu_state)
# menu.add_option("Single Player")
# menu.add_option("Multiplayer")
menu = Menu().self.menu

# # Set difficulty menu
# difficulty_menu = Menu(300, 200, 200, 100, "difficulty")
# difficulty_menu.add_option("Easy")
# difficulty_menu.add_option("Medium")
# difficulty_menu.add_option("Hard")

# Set up score
score = Score(PLAYER1_SCORE, PLAYER2_SCORE)

# Set up wall
WALL_WIDTH = screen_width
WALL_HEIGHT = 10
wall1 = Wall1(0,0, WALL_WIDTH, WALL_HEIGHT)
wall2 = Wall2(0, screen_height - WALL_HEIGHT, WALL_WIDTH, WALL_HEIGHT)

# Set up ball
ball_x = screen_width // 2
ball_y = screen_height // 2

ball = Ball(ball_x, ball_y, BALL_RADIUS, score)

# Set up paddles
paddle1_x = 20
paddle1_y = screen_height // 2 - PADDLE_HEIGHT // 2
paddle2_x = screen_width - 20 - PADDLE_WIDTH
paddle2_y = screen_height // 2 - PADDLE_HEIGHT // 2

pad1 = Paddle1(paddle1_x, paddle1_y, PADDLE_WIDTH, PADDLE_HEIGHT)
pad2 = Paddle2(paddle2_x, paddle2_y, PADDLE_WIDTH, PADDLE_HEIGHT)

# Create the background surface
background_surface = pygame.Surface((screen_width, screen_height))
Background_color(background_surface)

# Create the clock object to control the frame rate
clock = pygame.time.Clock()

# Menu loop
Menu_running = True
while Menu_running:
    window.blit(background_surface, (0, 0))

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    
    menu.draw(window)
    menu_keys = pygame.key.get_pressed()
    Menu_runnung,running = menu.handle_events(menu_keys)
    # Menu_runnung,running = menu.handle_events()


    pygame.display.flip()
    clock.tick(60)


# Game loop
# running = True
while running:
    # Clear the screen
    window.blit(background_surface, (0, 0))

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Draw walls
    wall1.draw(window)
    wall2.draw(window)

    # Draw the ball
    ball.move()
    ball.draw(window)

    # Draw the paddle
    keys = pygame.key.get_pressed()
    pad1.move(keys)
    pad1.draw(window)
    pad2.move(keys)
    pad2.draw(window)

    # Detect contact walls & paddles
    ball.contact_detect_wall(wall1)
    ball.contact_detect_pad1(pad1)
    ball.contact_detect_pad2(pad2)

    # Record score
    score.draw(window)

    # Update the window
    pygame.display.flip()
    
    # Limit the frame rate
    clock.tick(60)

pygame.quit()

TypeError: __init__() missing 4 required positional arguments: 'x', 'y', 'width', and 'height'

In [1]:
import pygame
import random

def Background_color(surface):
    for y in range(screen_height):
        for x in range(screen_width):
            ratio = (x - y + screen_width) / (2 * screen_width)
            r = int(WHITE[0] + (ADDFFF[0] - WHITE[0]) * ratio)
            g = int(WHITE[1] + (ADDFFF[1] - WHITE[1]) * ratio)
            b = int(WHITE[2] + (ADDFFF[2] - WHITE[2]) * ratio)
            surface.set_at((x,y), (r,g,b))

def Single_mode(difficulty):
    # if difficulty == "easy":
    #     ball.x_direction = random.choice((-3, 3))
    #     ball.x_direction = random.choice((-3, 3))
    # elif difficulty == "medium":
    #     ball.x_direction = random.choice((-4, 4))
    #     ball.x_direction = random.choice((-4, 4))
    # elif difficulty == "hard":
    #     ball.x_direction = random.choice((-10, 10))
    #     ball.x_direction = random.choice((-10, 10))
    single_running = True
    while single_running:
        # Clear the screen
        window.blit(background_surface, (0, 0))

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                single_running = False

        # Draw walls
        wall1.draw(window)
        wall2.draw(window)

        # Draw the ball
        ball.move()
        ball.draw(window)

        # Draw the paddle
        keys = pygame.key.get_pressed()
        pad1.move(keys)
        pad1.draw(window)
        pad2.single_mode(difficulty, ball)
        pad2.draw(window)

        # Detect contact walls & paddles
        ball.contact_detect_wall(wall1)
        ball.contact_detect_pad1(pad1)
        ball.contact_detect_pad2(pad2)

        # Record score
        score.draw(window)

        # Update the window
        pygame.display.flip()

        # Limit the frame rate
        clock.tick(60)
    pygame.quit()
    
def Multi_mode():
    running = True
    while running:
        # Clear the screen
        window.blit(background_surface, (0, 0))

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Draw walls
        wall1.draw(window)
        wall2.draw(window)

        # Draw the ball
        ball.move()
        ball.draw(window)

        # Draw the paddle
        keys = pygame.key.get_pressed()
        pad1.move(keys)
        pad1.draw(window)
        pad2.move(keys)
        pad2.draw(window)

        # Detect contact walls & paddles
        ball.contact_detect_wall(wall1)
        ball.contact_detect_pad1(pad1)
        ball.contact_detect_pad2(pad2)

        # Record score
        score.draw(window)

        # Update the window
        pygame.display.flip()

        # Limit the frame rate
        clock.tick(60)
    pygame.quit()

class Menu():
    def __init__(self, x, y, width, height, menu_state):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.options = []
        self.selected_option = 0

        self.menu = menu_state
        self.current_menu = "mode"
        self.current_menu_option = 0
        self.current_difficulty_option = 0

    def add_option(self, option):
        self.options.append(option)

    def draw(self, window):
        title_font = pygame.font.Font(None, 60)
        font = pygame.font.Font(None, 36)

        title_text = title_font.render("PongGame", True, CADETBLUE)
        title_rect = title_text.get_rect(center = (screen_width//2, screen_height//3))
        window.blit(title_text, title_rect)
        for i, option in enumerate(self.options):
            if self.current_menu_option  == i:
                pygame.draw.rect(window, CADETBLUE, 
                                 (self.x, 
                                  self.y + (i + 1) * (self.height // (len(self.options) + 1)) - 15,
                                  self.width, 30), 
                                  3)
            text = font.render(option, True, CADETBLUE)
            text_rect = text.get_rect(center=(self.x + self.width // 2, self.y + (i + 1) * (self.height // (len(self.options) + 1))))
            window.blit(text, text_rect)

    def handle_mouse_input(self, x, y):
        for i, option in enumerate(self.options):
            if self.x <= x <= self.x + self.width and self.y + (i + 1) * (self.height // (len(self.options) + 1)) <= y <= self.y + (i + 2) * (self.height // (len(self.options) + 1)):
                return i
        return None

    def handle_keyboard_input(self, keys):
        if keys[pygame.K_UP]:
            return -1
        elif keys[pygame.K_DOWN]:
            return 1
        else:
            return 0
    
    def handle_events(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return False, ""
            
            # Get mouse event
            elif event.type == pygame.MOUSEBUTTONDOWN:
                x, y = pygame.mouse.get_pos()
                if self.current_menu == "mode":
                    selected_option = self.handle_mouse_input(x, y)
                    if selected_option is not None:
                        if selected_option == 0:
                            self.current_menu = "difficulty"
                        elif selected_option == 1:
                            self.start_game()
                elif self.current_menu == "difficulty":
                    selected_option = self.handle_mouse_input(x, y)
                    if selected_option is not None:
                        if selected_option == 0:
                            Single_mode("easy")
                        elif selected_option == 1:
                            Single_mode("medium")
                        elif selected_option == 2:
                            Single_mode("hard")

            # Get keyboard event
            elif event.type == pygame.KEYDOWN:
                keys = pygame.key.get_pressed()
                if self.current_menu == "mode":
                    option_change = self.handle_keyboard_input(keys)
                    # print(option_change)
                    self.current_menu_option  += option_change
                    if self.current_menu_option  < 0:
                        # print("1")
                        self.current_menu_option  = len(self.options) - 1
                    elif self.current_menu_option  >= len(self.options):
                        # print("2")
                        self.current_menu_option  = 0

                    if keys[pygame.K_RETURN]:
                        if self.current_menu_option  == 0:
                            # print("3")
                            self.current_menu = "difficulty"
                            return False, "single"
                        elif self.current_menu_option  == 1:
                            Multi_mode()
                            # return False, "multi"
                            # print("4")
                            # self.start_game()
                    print("current:" + str(self.current_menu_option ) + "change:" + str(option_change))
                    
                elif self.current_menu == "difficulty":
                    option_change = self.handle_keyboard_input(keys)
                    
                    if option_change != 0:
                        self.current_difficulty_option += option_change
                        if self.current_difficulty_option  < 0:
                            # print("5")
                            self.current_difficulty_option  = len(self.options) - 1
                        elif self.current_difficulty_option  >= len(self.options):
                            # print("6")
                            self.current_difficulty_option  = 0
                    print("current:" + str(self.current_difficulty_option ) + "change:" + str(option_change))

                    if keys[pygame.K_RETURN]:
                        if self.current_difficulty_option  == 0:
                            Single_mode("easy")
                            print("easy")
                            # return False
                        elif self.current_difficulty_option  == 1:
                            Single_mode("medium")
                            print("medium")
                            # return False
                        elif self.current_difficulty_option  == 2:
                            Single_mode("hard")
                            print("hard")
                            # return False
        return True, ""

class Ball():
    def __init__(self, x, y, radius, score):
        self.x = x
        self.y = y
        self.x_direction = random.choice((-3, 3))
        self.y_direction = random.choice([-3, 3])
        self.radius = radius
        self.score = score
    
    def move(self):
        self.x += self.x_direction
        self.y += self.y_direction
        self.out_of_bound()
    
    def bounce(self, brick):
        if self.x < brick.x - (brick.radius / 1.414):
            self.x_direction = -2
        elif self.x > brick.x + (brick.radius / 1.414):
            self.x_direction = 2
        elif self.y < brick.y - (brick.radius / 1.414):
            self.y_direction = -2
        elif self.y > brick.y + (brick.radius / 1.414):
            self.y_direction = 2
    
    def contact_detect_wall(self, wall):
        if self.y - wall.height - self.radius  <= 0 or self.y >= screen_height - wall.height - self.radius:
            self.y_direction *= -1
    
    def contact_detect_pad1(self, pad):
        if self.x == pad.x + pad.width + self.radius and pad.y - self.radius <= self.y <= pad.y + pad.height + self.radius:
            self.x_direction = BALL_SPEED_X
        # if pad.x <= self.x <= pad.x + pad.width and (pad.y <= self.y + self.radius or pad.y + pad.height >= self.y - self.radius):
        #     self.y_direction *= -1

    def contact_detect_pad2(self, pad):        
        if self.x == pad.x - self.radius and pad.y <= self.y <= pad.y + pad.height:
            self.x_direction = -BALL_SPEED_X
        # if pad.x <= self.x <= pad.x + pad.width and pad.y <= self.y + self.radius:
        #     self.y_direction = -BALL_SPEED_Y
        # if pad.x <= self.x <= pad.x + pad.width and pad.y + pad.height >= self.y - self.radius:
        #     self.y_direction = BALL_SPEED_Y

    def out_of_bound(self):
        if self.x < 0:
            self.x = screen_width // 2
            self.y = screen_height * (random.choice([0.25, 0.375, 0.5, 0.625, 0.75]))
            self.x_direction *= random.choice([-1, 1])
            self.y_direction *= random.choice([-1, 1])
            self.score.player2_score += 1
        elif self.x + self.radius > screen_width:
            self.x = screen_width // 2
            self.y = screen_height * (random.choice([0.25, 0.375, 0.5, 0.625, 0.75]))
            self.x_direction *= random.choice([-1, 1])
            self.y_direction *= random.choice([-1, 1])
            self.score.player1_score += 1

    def contact_detect_brick(self, brick):
        if (brick.dis_to_brick(self.x, self.y) - self.radius <= 0) and brick.valid:
            self.bounce(brick)
            brick.valid = False
    
    def draw(self, window):
        pygame.draw.circle(window, DEEPSKYBLUE, (self.x, self.y), self.radius)

    def reset(self, x, y, x_direction, y_direction):
        self.x = x
        self.y = y
        self.x_direction = random.choice([-3, 3])
        self.y_direction = random.choice([-3, 3])

class Paddle1():
    def __init__(self, x, y, width, height):
        self.x = x
        self.y = y
        self.width = width
        self.height = height

    def draw(self, window):
        pygame.draw.rect(window, DEEPSKYBLUE, (self.x, self.y, self.width, self.height))

    def move(self, keys):
        if keys[pygame.K_w] and self.y - WALL_HEIGHT > 0:
            self.y -= PADDLE_SPEED
        if keys[pygame.K_s] and self.y < screen_height - self.height - WALL_HEIGHT:
            self.y += PADDLE_SPEED

class Paddle2(Paddle1):
    def move(self, keys):
        if keys[pygame.K_UP] and self.y - WALL_HEIGHT > 0:
            self.y -= PADDLE_SPEED
        if keys[pygame.K_DOWN] and self.y < screen_height - self.height - WALL_HEIGHT:
            self.y += PADDLE_SPEED

    def single_mode(self,difficulty, ball):
        if difficulty == "easy":
            speed = 0.5
        elif difficulty == "medium":
            speed = 1
        elif difficulty == "hard":
            speed = 3

        if self.y < ball.y and self.y < screen_height - self.height - WALL_HEIGHT:
            self.y += speed
        elif self.y + self.height > ball.y and self.y - WALL_HEIGHT > 0:
            self.y -= speed

class Wall1():
    def __init__(self, x, y, width, height):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
    def draw(self, window):
        pygame.draw.rect(window, CADETBLUE, (0, 0, WALL_WIDTH, WALL_HEIGHT))

class Wall2(Wall1):
    def draw(self, window):
        pygame.draw.rect(window, CADETBLUE, (0, screen_height - WALL_HEIGHT, WALL_WIDTH, WALL_HEIGHT))

class Score():
    def __init__(self, player1_score, player2_score):
        self.player1_score = player1_score
        self.player2_score = player2_score

    def draw(self, window):
        font = pygame.font.Font(None, 45)
        player1_score_text = font.render(str(self.player1_score), True, CADETBLUE)
        player1_score_rect = player1_score_text.get_rect(center=(screen_width // 4, 50))
        window.blit(player1_score_text, player1_score_rect)

        player2_score_text = font.render(str(self.player2_score), True, CADETBLUE)
        player2_score_rect = player2_score_text.get_rect(center=(3 * screen_width // 4, 50))
        window.blit(player2_score_text, player2_score_rect)

# Initialize Pygame
pygame.init()

# Define colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
CYAN =  (0, 255, 255)
ADDFFF = (173, 233, 255)
DEEPSKYBLUE = (0,191,255)
CADETBLUE = (95, 158, 160)

# Define game constants
BALL_RADIUS = 10
PADDLE_WIDTH = 10
PADDLE_HEIGHT = 60
BALL_SPEED_X = 3
BALL_SPEED_Y = 3
PADDLE_SPEED = 5
PLAYER1_SCORE = 0
PLAYER2_SCORE = 0

# Define game strings
Player1_name = "Player1"
Player2_name = "Player2"

# Set up the game window
screen_width = 800
screen_height = 400
window = pygame.display.set_mode((screen_width, screen_height))

# Set title
pygame.display.set_caption("Pong")

# Set player mode menu
Menu_state = "mode"
menu = Menu(300, 200, 200, 100, Menu_state)
menu.add_option("Single Player")
menu.add_option("Multiplayer")

# Set difficulty menu
difficulty_menu = Menu(300, 200, 200, 100, "difficulty")
difficulty_menu.add_option("Easy")
difficulty_menu.add_option("Medium")
difficulty_menu.add_option("Hard")

# Set up score
score = Score(PLAYER1_SCORE, PLAYER2_SCORE)

# Set up wall
WALL_WIDTH = screen_width
WALL_HEIGHT = 10
wall1 = Wall1(0,0, WALL_WIDTH, WALL_HEIGHT)
wall2 = Wall2(0, screen_height - WALL_HEIGHT, WALL_WIDTH, WALL_HEIGHT)

# Set up ball
ball_x = screen_width // 2
ball_y = screen_height // 2

ball = Ball(ball_x, ball_y, BALL_RADIUS, score)

# Set up paddles
paddle1_x = 20
paddle1_y = screen_height // 2 - PADDLE_HEIGHT // 2
paddle2_x = screen_width - 20 - PADDLE_WIDTH
paddle2_y = screen_height // 2 - PADDLE_HEIGHT // 2

pad1 = Paddle1(paddle1_x, paddle1_y, PADDLE_WIDTH, PADDLE_HEIGHT)
pad2 = Paddle2(paddle2_x, paddle2_y, PADDLE_WIDTH, PADDLE_HEIGHT)

# Create the background surface
background_surface = pygame.Surface((screen_width, screen_height))
Background_color(background_surface)

# Create the clock object to control the frame rate
clock = pygame.time.Clock()

# Play mode Menu loop
Menu_running = True
while Menu_running:
    window.blit(background_surface, (0, 0))

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            Menu_running = False
    
    menu.draw(window)
    # menu_keys = pygame.key.get_pressed()
    Menu_running, mode = menu.handle_events()
    pygame.display.flip()
    clock.tick(60)

print(mode)
# Difficulty Menu loop
if mode == "single":
    Difficulty_running = True
elif mode == "multi":
    Difficulty_running = False
    running = True

while Difficulty_running:
    window.blit(background_surface, (0, 0))

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            Difficulty_running = False
    
    difficulty_menu.draw(window)
    Difficulty_running = difficulty_menu.handle_events()
    pygame.display.flip()
    clock.tick(60)

if running == False:
    print("score")

pygame 2.3.0 (SDL 2.24.2, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html
current:1change:1
current:0change:1
current:1change:1
current:0change:1
single
current:1change:1
current:1change:0


error: video system not initialized

In [ ]:
import pygame
import random

def Background_color(surface):
    for y in range(screen_height):
        for x in range(screen_width):
            ratio = (x - y + screen_width) / (2 * screen_width)
            r = int(WHITE[0] + (ADDFFF[0] - WHITE[0]) * ratio)
            g = int(WHITE[1] + (ADDFFF[1] - WHITE[1]) * ratio)
            b = int(WHITE[2] + (ADDFFF[2] - WHITE[2]) * ratio)
            surface.set_at((x,y), (r,g,b))

class Menu():
    def __init__(self, x, y, width, height, menu_state):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.options = []
        self.selected_option = 0

        self.menu = menu_state
        self.current_menu = "mode"
        self.current_option = 0

    def add_option(self, option):
        self.options.append(option)

    def draw(self, window):
        title_font = pygame.font.Font(None, 60)
        font = pygame.font.Font(None, 36)

        title_text = title_font.render("PongGame", True, CADETBLUE)
        title_rect = title_text.get_rect(center = (screen_width//2, screen_height//3))
        window.blit(title_text, title_rect)
        for i, option in enumerate(self.options):
            if self.current_option == i:
                pygame.draw.rect(window, CADETBLUE, 
                                 (self.x, 
                                  self.y + (i + 1) * (self.height // (len(self.options) + 1)) - 15,
                                  self.width, 30), 
                                  3)
            text = font.render(option, True, CADETBLUE)
            text_rect = text.get_rect(center=(self.x + self.width // 2, self.y + (i + 1) * (self.height // (len(self.options) + 1))))
            window.blit(text, text_rect)

    # def draw(self, window):
    #     pygame.draw.rect(window, (255, 255, 255), (self.x, self.y, self.width, self.height), 2)
    #     font = pygame.font.Font(None, 36)

    #     for i, option in enumerate(self.options):
    #         text = font.render(option, True, (255, 255, 255))
    #         text_rect = text.get_rect(center=(self.x + self.width // 2, self.y + (i + 1) * (self.height // (len(self.options) + 1))))
    #         window.blit(text, text_rect)

    #         if self.selected_option == i:
    #             pygame.draw.rect(window, (255, 255, 255), (self.x, self.y + (i + 1) * (self.height // (len(self.options) + 1)), self.width, self.height // (len(self.options) + 1)), 2)

    def handle_mouse_input(self, x, y):
        for i, option in enumerate(self.options):
            if self.x <= x <= self.x + self.width and self.y + (i + 1) * (self.height // (len(self.options) + 1)) <= y <= self.y + (i + 2) * (self.height // (len(self.options) + 1)):
                return i
        return None

    def handle_keyboard_input(self, keys):
        if keys[pygame.K_UP]:
            return -1
        elif keys[pygame.K_DOWN]:
            return 1
        return 0
    
    def handle_events(self, keys):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return False, True
            
            # Get mouse event
            elif event.type == pygame.MOUSEBUTTONDOWN:
                x, y = pygame.mouse.get_pos()
                if self.current_menu == self.menu:
                    selected_option = self.handle_mouse_input(x, y)
                    if selected_option is not None:
                        if selected_option == 0:
                            self.current_menu = "difficulty"
                        elif selected_option == 1:
                            self.start_game()
                elif self.current_menu == "difficulty":
                    selected_option = self.handle_mouse_input(x, y)
                    if selected_option is not None:
                        if selected_option == 0:
                            self.start_game(difficulty="easy")
                        elif selected_option == 1:
                            self.start_game(difficulty="medium")
                        elif selected_option == 2:
                            self.start_game(difficulty="hard")

            # Get keyboard event
            elif event.type == pygame.KEYDOWN:
                if self.current_menu == self.menu:
                    option_change = self.handle_keyboard_input(keys)
                    self.current_option += option_change
                    if self.current_option < 0:
                        print("1")
                        self.current_option = len(self.options) - 1
                    elif self.current_option >= len(self.options):
                        print("2")
                        self.current_option = 0
                    if keys[pygame.K_RETURN]:
                        if self.current_option == 0:
                            print("3")
                            self.current_menu = "difficulty"
                        elif self.current_option == 1:
                            print("4")
                            self.start_game()
                elif self.current_menu == "difficulty":
                    option_change = self.handle_keyboard_input(keys)
                    if option_change != 0:
                        self.current_option += option_change
                        if self.current_option < 0:
                            print("5")
                            self.current_option = len(self.options) - 1
                        elif self.current_option >= len(self.options):
                            print("6")
                            self.current_option = 0
                    if keys[pygame.K_RETURN]:
                        if self.current_option == 0:
                            self.start_game(difficulty="easy")
                        elif self.current_option == 1:
                            self.start_game(difficulty="medium")
                        elif self.current_option == 2:
                            self.start_game(difficulty="hard")
        return False, True
    
    def start_game(self, difficulty):
        return

class Ball():
    def __init__(self, x, y, radius, score):
        self.x = x
        self.y = y
        self.x_direction = random.choice((-3, 3))
        self.y_direction = random.choice([-3, 3])
        self.radius = radius
        self.score = score
    
    def move(self):
        self.x += self.x_direction
        self.y += self.y_direction
        self.out_of_bound()
    
    def bounce(self, brick):
        if self.x < brick.x - (brick.radius / 1.414):
            self.x_direction = -2
        elif self.x > brick.x + (brick.radius / 1.414):
            self.x_direction = 2
        elif self.y < brick.y - (brick.radius / 1.414):
            self.y_direction = -2
        elif self.y > brick.y + (brick.radius / 1.414):
            self.y_direction = 2
    
    def contact_detect_wall(self, wall):
        if self.y - wall.height - self.radius  <= 0 or self.y >= screen_height - wall.height - self.radius:
            self.y_direction *= -1
    
    def contact_detect_pad1(self, pad):
        if self.x == pad.x + pad.width + self.radius and pad.y - self.radius <= self.y <= pad.y + pad.height + self.radius:
            self.x_direction = BALL_SPEED_X
        # if pad.x <= self.x <= pad.x + pad.width and (pad.y <= self.y + self.radius or pad.y + pad.height >= self.y - self.radius):
        #     self.y_direction *= -1

    def contact_detect_pad2(self, pad):        
        if self.x == pad.x - self.radius and pad.y <= self.y <= pad.y + pad.height:
            self.x_direction = -BALL_SPEED_X
        # if pad.x <= self.x <= pad.x + pad.width and pad.y <= self.y + self.radius:
        #     self.y_direction = -BALL_SPEED_Y
        # if pad.x <= self.x <= pad.x + pad.width and pad.y + pad.height >= self.y - self.radius:
        #     self.y_direction = BALL_SPEED_Y

    def out_of_bound(self):
        if self.x < 0:
            self.x = screen_width // 2
            self.y = screen_height * (random.choice([0.25, 0.375, 0.5, 0.625, 0.75]))
            self.x_direction *= random.choice([-1, 1])
            self.y_direction *= random.choice([-1, 1])
            self.score.player2_score += 1
        elif self.x + self.radius > screen_width:
            self.x = screen_width // 2
            self.y = screen_height * (random.choice([0.25, 0.375, 0.5, 0.625, 0.75]))
            self.x_direction *= random.choice([-1, 1])
            self.y_direction *= random.choice([-1, 1])
            self.score.player1_score += 1

    def contact_detect_brick(self, brick):
        if (brick.dis_to_brick(self.x, self.y) - self.radius <= 0) and brick.valid:
            self.bounce(brick)
            brick.valid = False
    
    def draw(self, window):
        pygame.draw.circle(window, DEEPSKYBLUE, (self.x, self.y), self.radius)

    def reset(self, x, y, x_direction, y_direction):
        self.x = x
        self.y = y
        self.x_direction = random.choice([-3, 3])
        self.y_direction = random.choice([-3, 3])

class Paddle1():
    def __init__(self, x, y, width, height):
        self.x = x
        self.y = y
        self.width = width
        self.height = height

    def draw(self, window):
        pygame.draw.rect(window, DEEPSKYBLUE, (self.x, self.y, self.width, self.height))

    def move(self, keys):
        if keys[pygame.K_w] and self.y - WALL_HEIGHT > 0:
            self.y -= PADDLE_SPEED
        if keys[pygame.K_s] and self.y < screen_height - self.height - WALL_HEIGHT:
            self.y += PADDLE_SPEED

class Paddle2(Paddle1):
    def move(self, keys):
        if keys[pygame.K_UP] and self.y - WALL_HEIGHT > 0:
            self.y -= PADDLE_SPEED
        if keys[pygame.K_DOWN] and self.y < screen_height - self.height - WALL_HEIGHT:
            self.y += PADDLE_SPEED

class Wall1():
    def __init__(self, x, y, width, height):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
    def draw(self, window):
        pygame.draw.rect(window, CADETBLUE, (0, 0, WALL_WIDTH, WALL_HEIGHT))

class Wall2(Wall1):
    def draw(self, window):
        pygame.draw.rect(window, CADETBLUE, (0, screen_height - WALL_HEIGHT, WALL_WIDTH, WALL_HEIGHT))

class Score():
    def __init__(self, player1_score, player2_score):
        self.player1_score = player1_score
        self.player2_score = player2_score

    def draw(self, window):
        font = pygame.font.Font(None, 45)
        player1_score_text = font.render(str(self.player1_score), True, CADETBLUE)
        player1_score_rect = player1_score_text.get_rect(center=(screen_width // 4, 50))
        window.blit(player1_score_text, player1_score_rect)

        player2_score_text = font.render(str(self.player2_score), True, CADETBLUE)
        player2_score_rect = player2_score_text.get_rect(center=(3 * screen_width // 4, 50))
        window.blit(player2_score_text, player2_score_rect)

# Initialize Pygame
pygame.init()

# Define colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
CYAN =  (0, 255, 255)
ADDFFF = (173, 233, 255)
DEEPSKYBLUE = (0,191,255)
CADETBLUE = (95, 158, 160)

# Define game constants
BALL_RADIUS = 10
PADDLE_WIDTH = 10
PADDLE_HEIGHT = 60
BALL_SPEED_X = 3
BALL_SPEED_Y = 3
PADDLE_SPEED = 5
PLAYER1_SCORE = 0
PLAYER2_SCORE = 0

# Define game strings
Player1_name = "Player1"
Player2_name = "Player2"

# Set up the game window
screen_width = 800
screen_height = 400
window = pygame.display.set_mode((screen_width, screen_height))

# Set title
pygame.display.set_caption("Pong")

# Set player mode menu
Menu_state = "mode"
menu = Menu(300, 200, 200, 100, Menu_state)
menu.add_option("Single Player")
menu.add_option("Multiplayer")

# Set difficulty menu
difficulty_menu = Menu(300, 200, 200, 100, "difficulty")
difficulty_menu.add_option("Easy")
difficulty_menu.add_option("Medium")
difficulty_menu.add_option("Hard")

# Set up score
score = Score(PLAYER1_SCORE, PLAYER2_SCORE)

# Set up wall
WALL_WIDTH = screen_width
WALL_HEIGHT = 10
wall1 = Wall1(0,0, WALL_WIDTH, WALL_HEIGHT)
wall2 = Wall2(0, screen_height - WALL_HEIGHT, WALL_WIDTH, WALL_HEIGHT)

# Set up ball
ball_x = screen_width // 2
ball_y = screen_height // 2

ball = Ball(ball_x, ball_y, BALL_RADIUS, score)

# Set up paddles
paddle1_x = 20
paddle1_y = screen_height // 2 - PADDLE_HEIGHT // 2
paddle2_x = screen_width - 20 - PADDLE_WIDTH
paddle2_y = screen_height // 2 - PADDLE_HEIGHT // 2

pad1 = Paddle1(paddle1_x, paddle1_y, PADDLE_WIDTH, PADDLE_HEIGHT)
pad2 = Paddle2(paddle2_x, paddle2_y, PADDLE_WIDTH, PADDLE_HEIGHT)

# Create the background surface
background_surface = pygame.Surface((screen_width, screen_height))
Background_color(background_surface)

# Create the clock object to control the frame rate
clock = pygame.time.Clock()

# Menu loop
Menu_running = True
while Menu_running:
    window.blit(background_surface, (0, 0))

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    
    menu.draw(window)
    menu_keys = pygame.key.get_pressed()
    Menu_runnung, running = menu.handle_events(pygame.key.get_pressed())
    pygame.display.flip()
    clock.tick(60)

# Game loop
running = True
while running:
    # Clear the screen
    window.blit(background_surface, (0, 0))

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Draw walls
    wall1.draw(window)
    wall2.draw(window)

    # Draw the ball
    ball.move()
    ball.draw(window)

    # Draw the paddle
    keys = pygame.key.get_pressed()
    pad1.move(keys)
    pad1.draw(window)
    pad2.move(keys)
    pad2.draw(window)

    # Detect contact walls & paddles
    ball.contact_detect_wall(wall1)
    ball.contact_detect_pad1(pad1)
    ball.contact_detect_pad2(pad2)

    # Record score
    score.draw(window)

    # Update the window
    pygame.display.flip()
    
    # Limit the frame rate
    clock.tick(60)

pygame.quit()